### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from astropy.io import fits
from survey_tools import catalog
from survey_tools.utility import plot

class StructType:
    pass

### Options

In [ ]:
field = 'COSMOS'
redshift_ranges = np.array([[0.7,1.1],[1.1,1.8],[1.8,2.7]])
min_line_snr = 2.0

match field:
    case 'COSMOS':
        ra_min = 149.37302
        ra_max = 150.77901
        dec_min = 1.6040021
        dec_max = 2.8099799

plot_mode = 'default'
plot_colours = plot.get_plot_colours(plot_mode)

### Read Catalog

In [ ]:
sources_hdul = fits.open(f"{catalog.get_default_data_path()}/catalogs/DESI/edr_galaxy_stellarmass_lineinfo_v1.0.fits")
DESI_data = sources_hdul[1].data

num_galaxies = len(DESI_data)
print(f"Num of Galaxies: {num_galaxies:,}")

### Create Filters

In [ ]:
zbin = np.zeros((num_galaxies), dtype=np.int_)
for i in np.arange(len(redshift_ranges)):
    zbin[(DESI_data['Z'] >= redshift_ranges[i,0]) & (DESI_data['Z'] <  redshift_ranges[i,1])] = i + 1

filters = StructType()

filters.usable = DESI_data['Z'] > 0.0
num_usable = np.sum(filters.usable)

filters.field = filters.usable & (DESI_data['TARGET_RA'] >= ra_min) & (DESI_data['TARGET_RA'] <= ra_max) & (DESI_data['TARGET_DEC'] >= dec_min) & (DESI_data['TARGET_DEC'] <= dec_max)
num_field = np.sum(filters.field)

filters.redshift = filters.field & (zbin > 0)
num_redshift = np.sum(filters.redshift)

has_OIIa = filters.redshift & (DESI_data['OII3726_FLUX'] > 0) & (DESI_data['OII3726_FLUXERR'] > 0)
snr_OIIa = DESI_data['OII3726_FLUX'][has_OIIa] / DESI_data['OII3726_FLUXERR'][has_OIIa]
good_snr_OIIa = snr_OIIa > min_line_snr

has_OIIb = filters.redshift & (DESI_data['OII3729_FLUX'] > 0) & (DESI_data['OII3729_FLUXERR'] > 0)
snr_OIIb = DESI_data['OII3729_FLUX'][has_OIIb] / DESI_data['OII3729_FLUXERR'][has_OIIb]
good_snr_OIIb = snr_OIIb > min_line_snr

has_Hb = filters.redshift & (DESI_data['HBETA_FLUX'] > 0) & (DESI_data['HBETA_FLUXERR'] > 0)
snr_Hb = DESI_data['HBETA_FLUX'][has_Hb] / DESI_data['HBETA_FLUXERR'][has_Hb]
good_snr_Hb = snr_Hb > min_line_snr

has_OIIIa = filters.redshift & (DESI_data['OIII4959_FLUX'] > 0) & (DESI_data['OIII4959_FLUXERR'] > 0)
snr_OIIIa = DESI_data['OIII4959_FLUX'][has_OIIIa] / DESI_data['OIII4959_FLUXERR'][has_OIIIa]
good_snr_OIIIa = snr_OIIIa > min_line_snr

has_OIIIb = filters.redshift & (DESI_data['OIII5007_FLUX'] > 0) & (DESI_data['OIII5007_FLUXERR'] > 0)
snr_OIIIb = DESI_data['OIII5007_FLUX'][has_OIIIb] / DESI_data['OIII5007_FLUXERR'][has_OIIIb]
good_snr_OIIIb = snr_OIIIb > min_line_snr

has_NIIa = filters.redshift & (DESI_data['NII6548_FLUX'] > 0) & (DESI_data['NII6548_FLUXERR'] > 0)
snr_NIIa = DESI_data['NII6548_FLUX'][has_NIIa] / DESI_data['NII6548_FLUXERR'][has_NIIa]
good_snr_NIIa = snr_NIIa > min_line_snr

has_Ha = filters.redshift & (DESI_data['HALPHA_FLUX'] > 0) & (DESI_data['HALPHA_FLUXERR'] > 0)
snr_Ha = DESI_data['HALPHA_FLUX'][has_Ha] / DESI_data['HALPHA_FLUXERR'][has_Ha]
good_snr_Ha = snr_Ha > min_line_snr

has_NIIb = filters.redshift & (DESI_data['NII6583_FLUX'] > 0) & (DESI_data['NII6583_FLUXERR'] > 0)
snr_NIIb = DESI_data['NII6583_FLUX'][has_NIIb] / DESI_data['NII6583_FLUXERR'][has_NIIb]
good_snr_NIIb = snr_NIIb > min_line_snr

has_SIIa = filters.redshift & (DESI_data['SII6716_FLUX'] > 0) & (DESI_data['SII6716_FLUXERR'] > 0)
snr_SIIa = DESI_data['SII6716_FLUX'][has_SIIa] / DESI_data['SII6716_FLUXERR'][has_SIIa]
good_snr_SIIa = snr_SIIa > min_line_snr

has_SIIb = filters.redshift & (DESI_data['SII6731_FLUX'] > 0) & (DESI_data['SII6731_FLUXERR'] > 0)
snr_SIIb = DESI_data['SII6731_FLUX'][has_SIIb] / DESI_data['SII6731_FLUXERR'][has_SIIb]
good_snr_SIIb = snr_SIIb > min_line_snr

### Galaxies

In [ ]:
print(f"Num Usable Galaxies: {num_usable}")
print(f"Num Usable Galaxies in Field: {num_field}")
print(f"Num Usable Galaxies in Field and Redshift Range: {num_redshift}")

_, ax = plot.create_plot(plot_mode, title='DESI EDR Galaxies')
values, bins, _ = ax.hist(DESI_data['Z'][filters.usable], int(np.sqrt(num_galaxies)), label="Usable", rwidth=0.95, color=plot_colours.colour2)
ax.hist(DESI_data['Z'][filters.field], int(np.sqrt(num_field)), label=field, rwidth=0.95, color=plot_colours.colour4)
ax.hist(DESI_data['Z'][filters.redshift], int(np.sqrt(num_redshift)), rwidth=0.95, label="Redshift", color=plot_colours.colour6)
plot.format_hist(ax, values=values, bins=bins, xtickspacing=0.5, ytickspacing=100)
ax.set_xlabel('$z$')
ax.set_ylabel('Galaxies')
#ax.set_xlim([params.min_galaxy_z, params.max_galaxy_z])
ax.legend(loc='upper center', markerscale=2.0, ncol=6)

_, ax = plot.create_plot(plot_mode, title=f"DESI EDR Galaxies")
for i in np.arange(np.size(redshift_ranges, axis=0)):
    redshift_filter = filters.field & (zbin == i+1) & (DESI_data['SED_MASS'] > 0) & (DESI_data['SED_SFR'] > 0)
    ax.scatter(np.log10(DESI_data['SED_MASS'][redshift_filter]), np.log10(DESI_data['SED_SFR'][redshift_filter]), s=15, marker='o', edgecolors='black', facecolors=plot_colours.colours[(i+1)*2], alpha=0.75, linewidths=0.5, label=f"${redshift_ranges[i,0]} \\geq z \\leq {redshift_ranges[i,1]}$")
ax.set_xlabel('Mass [log $M_*/M_\\odot$]')
ax.set_ylabel('SFR [log SFR/$M_\\odot$/year]')
ax.legend(loc='lower right')

_, ax = plot.create_plot(plot_mode, title=f"DESI EDR Redshift Errors")
ax.scatter(DESI_data['Z'][filters.redshift], DESI_data['ZERR'][filters.redshift], s=5, marker='o', edgecolors='none', facecolors=plot_colours.colour6)
ax.set_xlabel('$z$')
ax.set_ylabel('$\\sigma_z$')

_, ax = plot.create_plot(plot_mode, title=f"DESI EDR Redshift Errors")
dz = np.mean(DESI_data['ZERR'][filters.redshift])
print(f"Mean Redshift Error: {dz:.5f}")
values, bins, _ = ax.hist(DESI_data['ZERR'][filters.redshift], bins=50, rwidth=0.95, color=plot_colours.colour6)
plot.format_hist(ax, values=values, bins=bins, xtickspacing=0.001, ytickspacing=100)
ax.axvline(dz, color='red', linestyle='-', linewidth=1)
ax.set_xlim([0, 0.0004])
ax.set_xlabel('$\\sigma_z$/(1+z)')
ax.set_ylabel('Galaxies')
ax.text(0.95, 0.9, f"Mean: {dz:.5f}", transform=ax.transAxes, ha='right', bbox=dict(facecolor='white', alpha=0.5))

del ax, values, bins, redshift_filter

### Emission Lines

In [ ]:
print('Emission Line Measurements:')
print(f"  OIIa: {np.sum(good_snr_OIIa)}/{len(snr_OIIa)}")
print(f"  OIIb: {np.sum(good_snr_OIIb)}/{len(snr_OIIb)}")
print(f"    Hb: {np.sum(good_snr_Hb)}/{len(snr_Hb)}")
print(f" OIIIa: {np.sum(good_snr_OIIIa)}/{len(snr_OIIIa)}")
print(f" OIIIb: {np.sum(good_snr_OIIIb)}/{len(snr_OIIIb)}")
print(f"  NIIa: {np.sum(good_snr_NIIa)}/{len(snr_NIIa)}")
print(f"    Ha: {np.sum(good_snr_Ha)}/{len(snr_Ha)}")
print(f"  NIIb: {np.sum(good_snr_NIIb)}/{len(snr_NIIb)}")
print(f"  SIIa: {np.sum(good_snr_SIIa)}/{len(snr_SIIa)}")
print(f"  SIIb: {np.sum(good_snr_SIIb)}/{len(snr_SIIb)}")
print(f"   Any: {np.sum(has_OIIa | has_OIIb | has_Hb | has_OIIIa | has_OIIIb | has_NIIa | has_Ha | has_NIIb | has_SIIa | has_SIIb)}")

_, ax = plot.create_plot(plot_mode, title=f"DESI Emission Lines")
ax.scatter(DESI_data['Z'][has_OIIa][good_snr_OIIa], snr_OIIa[good_snr_OIIa], label="OIIa", s=5, marker='o', edgecolors='none', facecolors=plot_colours.colour7)
ax.scatter(DESI_data['Z'][has_OIIb][good_snr_OIIb], snr_OIIb[good_snr_OIIb], label="OIIb", s=5, marker='o', edgecolors='none', facecolors=plot_colours.colour6)
ax.scatter(DESI_data['Z'][has_Hb][good_snr_Hb], snr_Hb[good_snr_Hb], label="Hb", s=5, marker='o', edgecolors='none', facecolors=plot_colours.colour5)
ax.scatter(DESI_data['Z'][has_OIIIa][good_snr_OIIIa], snr_OIIIa[good_snr_OIIIa], label="OIIIa", s=5, marker='o', edgecolors='none', facecolors=plot_colours.colour4)
ax.scatter(DESI_data['Z'][has_OIIIb][good_snr_OIIIb], snr_OIIIb[good_snr_OIIIb], label="OIIIb", s=5, marker='o', edgecolors='none', facecolors=plot_colours.colour3)
ax.set_xlabel('$z$')
ax.set_ylabel('SNR')
ax.legend(loc='upper right')

del ax